## Movie Review Classification

![This is getting exciting](https://i.kinja-img.com/gawker-media/image/upload/s--hIgTSFEs--/c_fit,fl_progressive,q_80,w_320/17j2zn73qxdlfgif.jpg)

In [1]:
from nltk.corpus import movie_reviews as reviews

X = [reviews.raw(fileid) for fileid in reviews.fileids()]
y = [reviews.categories(fileid)[0] for fileid in reviews.fileids()]

1 - Print a positive and negative review:

In [2]:
pos_review =[]
neg_review =[]

for i in y:
    if  i == "pos":
        pos_review.append(X[y.index(i)])
    
    if  i == "neg":
        neg_review.append(X[y.index(i)])

In [3]:
#Positive review

print("Positive review :") 
print(set(pos_review))

Positive review :
{'films adapted from comic books have had plenty of success , whether they\'re about superheroes ( batman , superman , spawn ) , or geared toward kids ( casper ) or the arthouse crowd ( ghost world ) , but there\'s never really been a comic book like from hell before . \nfor starters , it was created by alan moore ( and eddie campbell ) , who brought the medium to a whole new level in the mid \'80s with a 12-part series called the watchmen . \nto say moore and campbell thoroughly researched the subject of jack the ripper would be like saying michael jackson is starting to look a little odd . \nthe book ( or " graphic novel , " if you will ) is over 500 pages long and includes nearly 30 more that consist of nothing but footnotes . \nin other words , don\'t dismiss this film because of its source . \nif you can get past the whole comic book thing , you might find another stumbling block in from hell\'s directors , albert and allen hughes . \ngetting the hughes brothers 

In [4]:
#Negative review

print("Negative review :") 
print(set(neg_review))

Negative review :
{'plot : two teen couples go to a church party , drink and then drive . \nthey get into an accident . \none of the guys dies , but his girlfriend continues to see him in her life , and has nightmares . \nwhat\'s the deal ? \nwatch the movie and " sorta " find out . . . \ncritique : a mind-fuck movie for the teen generation that touches on a very cool idea , but presents it in a very bad package . \nwhich is what makes this review an even harder one to write , since i generally applaud films which attempt to break the mold , mess with your head and such ( lost highway & memento ) , but there are good and bad ways of making all types of films , and these folks just didn\'t snag this one correctly . \nthey seem to have taken this pretty neat concept , but executed it terribly . \nso what are the problems with the movie ? \nwell , its main problem is that it\'s simply too jumbled . \nit starts off " normal " but then downshifts into this " fantasy " world in which you , a

2 - Using the scikit train_test_split function (http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html), split the data into a training set and a test set. 

In [5]:
import numpy as np
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)


3 - Then lemmatize or stem the reviews, and transform the documents to tf-idf.

In [6]:
#Using Porter stemmer

# Cleaning the texts
import re
import nltk
#nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
corpus = []
for i in range(0, len(X)):
    review = re.sub('[^a-zA-Z]', ' ', X[i])
    review = review.lower()
    review = review.split()
    ps = PorterStemmer()
    review = [ps.stem(word) for word in review if not word in set(stopwords.words('english'))]
    review = ' '.join(review)
    corpus.append(review)

In [7]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vectorizer = TfidfVectorizer(max_df=0.8, max_features=200000,
                                 min_df=0.2, stop_words='english',
                                 use_idf=True, ngram_range=(1,3))

tfidf_matrix = tfidf_vectorizer.fit_transform(corpus) 



In [8]:
# Splitting the dataset into the Training set and Test set
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(tfidf_matrix, y, test_size = 0.20, random_state = 0)



4 - Finally, build a model. To start, use a logistic regression (which we will review in detail in the coming lectures) (http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html)

In [9]:
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder

#Encoding the y label for training and test set
labelencoder_X = LabelEncoder()
y_train = labelencoder_X.fit_transform(y_train)
y_test = labelencoder_X.fit_transform(y_test)

#Logistic regression
clf_A = LogisticRegression(penalty='l2') 
clf_A.fit(X_train, y_train)    #Training Logistic regression model 


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [10]:
y_pred_A = clf_A.predict(X_test)

In [11]:
# Making the Confusion Matrix
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_pred_A, y_test)
cm 

array([[145,  38],
       [ 55, 162]])

5 - Measure the efficacy of your model using the Reciever Operator Characteristic (ROC) Area Under the Curve (AUC). Report this metric on the test set of your data.

For more info on this, see: http://scikit-learn.org/stable/auto_examples/model_selection/plot_roc_crossval.html#sphx-glr-auto-examples-model-selection-plot-roc-crossval-py

In [12]:
#code to retrieve the ROC and AUC
from sklearn.metrics import roc_curve,auc 

#function to plot ROC and AUC
def roc_auc(y_test,y_pred):
    fpr,tpr,threshold = roc_curve(y_test, y_pred) 
    roc_auc = auc(fpr,tpr)
    return(roc_auc)


In [13]:
#roc and auc
roc_auc_result = roc_auc(y_test, y_pred_A)

6 - Change a parameter in your model (introduce regularization) or change a parameter in your word vector transformation (http://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html). Try introducing the use of stop words, or employing a cutoff on terms with min or max df.

In [15]:
#Models with different. parameters
from sklearn.svm import SVC

clf_B = SVC(kernel='rbf')
clf_C = LogisticRegression(penalty='l1')
clf_D = LogisticRegression(penalty='l2')
clf_E = SVC(kernel='linear')
 


7 - Make four models in total, changing parameters and comparing the AUC results. Report your findings in a tabular form.

In [16]:
clf_B.fit(X_train, y_train)
clf_C.fit(X_train, y_train)
clf_D.fit(X_train, y_train)
clf_E.fit(X_train, y_train)


SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='linear',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [17]:
y_pred_B = clf_B.predict(X_test)
y_pred_C = clf_C.predict(X_test)
y_pred_D = clf_D.predict(X_test)
y_pred_E = clf_E.predict(X_test)

In [18]:

roc_auc_1 = roc_auc(y_test, y_pred_B)  

roc_auc_2 = roc_auc(y_test, y_pred_C)

roc_auc_3 = roc_auc(y_test, y_pred_D)

roc_auc_4 = roc_auc(y_test, y_pred_E)


In [19]:
import pandas as pd
d = {'Index': [1, 2, 3, 4], 'Model features': ['Model: SVM , Kernel: rbf  , Stemmer : Porter' ,  
                                               'Model: Logistic Regression , Penalty: L1, Stemmer : Porter',
                                               'Model: Logistic Regression , Penalty: L2, Stemmer : Porter' ,
                                               'Model: SVM , Kernel: Linear , Stemmer : Porter'], 'Roc and Auc': [roc_auc_1, roc_auc_2 ,roc_auc_3 , roc_auc_4]}
df = pd.DataFrame(data=d)

In [20]:
df.set_index('Index')

,Model features,Roc and Auc
Index,,
1,"Model: SVM , Kernel: rbf , Stemmer : Porter",0.7100
2,"Model: Logistic Regression , Penalty: L1, Stem...",0.7600
3,"Model: Logistic Regression , Penalty: L2, Stem...",0.7675
4,"Model: SVM , Kernel: Linear , Stemmer : Porter",0.7575
